In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cellimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e8e8cedb0b794dd09dd689eda767e8e5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yAjzdllw-GtADEDB-62koF1BPpBKD7jOoKexKrZ7wXk4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_e8e8cedb0b794dd09dd689eda767e8e5.get_object(Bucket='lifeexpectancy-donotdelete-pr-zgyutdpgg2ouk4',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df = pd.read_csv(body)
df.head()

In [ ]:
df=df.drop('Year',axis=1)

In [ ]:
df.head()

In [ ]:
status=pd.get_dummies(df['Status'],drop_first=True)

In [ ]:
df=pd.concat([df,status],axis=1)
df.drop('Status',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
x=df.drop(['Life expectancy ','Country'],axis=1)
y=df['Life expectancy ']
x.fillna(value=x.mean(),inplace=True)
y.fillna(value=y.mean(),inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge
lm=LinearRegression()

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
lm.score(x_train,y_train)

In [ ]:
pd=lm.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,make_scorer

In [ ]:
print('Coefficients: \n', lm.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test,pd))
print("Mean absolute error: %.2f"
      % mean_absolute_error(y_test,pd))
print('R_square score: %.2f' % r2_score(y_test,pd))

In [ ]:
from sklearn.model_selection import GridSearchCV
scoring = make_scorer(r2_score)
grid_cv = GridSearchCV(Ridge(),
              param_grid={'alpha': range(0, 10), 'max_iter' : [10, 100, 1000]},
              scoring=scoring, cv=5, refit=True)

In [ ]:
grid_cv.fit(x_train, y_train)
print("Best Parameters: " + str(grid_cv.best_params_))
result = grid_cv.cv_results_

In [ ]:
pd1=grid_cv.best_estimator_.predict(x_test)

In [ ]:

print("Mean squared error: %.2f"
      % mean_squared_error(y_test,pd1))
print("Mean absolute error: %.2f"
      % mean_absolute_error(y_test,pd1))
print('R_square score: %.2f' % r2_score(y_test,pd1))

In [ ]:
!pip install watson-machine-learning-client

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials={
  "apikey": "enter your api key",
  "instance_id": "enter your instance id",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}


In [ ]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "enter name", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "enter mail", 
               client.repository.ModelMetaNames.NAME: "enter model name"}

In [ ]:
model_artifact =client.repository.store_model(grid_cv.best_estimator_, meta_props=model_props)

In [ ]:
published_model_uid = client.repository.get_model_uid(model_artifact)
published_model_uid

In [ ]:
deployment = client.deployments.create(published_model_uid, name="life_expectancy")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)

In [ ]:
scoring_endpoint